In [ ]:
%run ../notebook_preamble.ipy

from copy import deepcopy
import joblib
import seaborn as sns
import pymc3 as pm
import scipy.stats as ss
import theano.tensor as tt
from snepits.models.models_spec import *
from snepits.models.pymc_wrapper import PymcWrapper, PymcWrapperGrad

from pymc3.step_methods import SMC

# SIS

In [ ]:
m = SIS_pop
sizes = np.ones(100, dtype=np.int) * 5
params = np.array([2, 0.1])

model = m(sizes=sizes, params=params, data=None)

In [ ]:
loglike = PymcWrapper(model)

ndraws = 500
nburn = 500

with pm.Model() as pm_model:
    m = pm.HalfNormal('m', sd=5)
    c = pm.Uniform('c', lower=0., upper=10.)

    theta = tt.as_tensor_variable([m, c])

#     pm.DensityDist('likelihood', lambda v: loglike(v), observed={'v': theta})
    pm.Potential('likelihood', loglike(theta))

    trace = pm.sample(ndraws, tune=nburn, discard_tuned_samples=True)

_ = pm.traceplot(trace, lines={'m': model.t_params[0], 'c': model.t_params[1]})

# SIS ACR

In [ ]:
X = np.linspace(0, 10)
plt.plot(X, np.array([[ss.halfnorm(scale=i).pdf(x) for x in X] for i in [2, 5, 10]]).T);

In [ ]:
m = SIS_ACR_pop_gen
params = np.array([1, 1, 2, 1.5, 1, 1, 0.1])
sizes = np.ones((100, 3), dtype=np.int) * 2
sizes[:, 0] = sizes[:, 1:].sum(1)

model = m(SIS_ACR_orthogonal_reparam, sizes=sizes, params=params, data=None)

In [ ]:
ndraws = 30
nburn = 10

with pm.Model() as pm_model:
    beta_c = pm.HalfNormal('beta_c', sd=2)
    beta_l = pm.HalfNormal('beta_l', sd=2)
    beta_h = pm.HalfNormal('beta_h', sd=2)
    rho_c = pm.HalfNormal('rho_c', sd=2)
    rho_h = pm.HalfNormal('rho_h', sd=2)
    g = pm.HalfNormal('g', sd=2)
    eps = pm.HalfNormal('eps', sd=2)

    theta = tt.as_tensor_variable([beta_c, beta_l, beta_h, rho_c, rho_h, g, eps])

    loglike = PymcWrapper(deepcopy(model))
#     pm.DensityDist('likelihood', lambda v: loglike(v), observed={'v': theta})
    pm.Potential('likelihood', loglike(theta))

#     trace = pm.sample(ndraws, tune=nburn, discard_tuned_samples=True, chains=1)
    trace = pm.sample(step=SMC(), draws=1000, chains=4)

# plot the traces
_ = pm.traceplot(trace, lines={k: v for k, v in zip(model.param_l, model.t_params)})

# put the chains in an array (for later!)
df_trace = pm.trace_to_dataframe(trace)

In [ ]:
# half normal
print(loglike.call_counter, loglike.logpgrad.call_counter)
print(loglike.model.call_counter, loglike.model.sub_pops[0].call_counter)

In [ ]:
pm.summary(trace)

In [ ]:
sns.heatmap(pd.DataFrame(pm.trace_cov(trace, model=pm_model),
                         columns=model.param_l, index=model.param_l),
            annot=True, fmt='1.2g')

In [ ]:
def pairplot_truth(df_trace):
    ax = sns.pairplot(df_trace, diag_kind='kde')

    if hasattr(model, 't_params'):
        c = 'r'
        D = len(model.t_params)
        for i in range(D):
            ax.axes[i][i].axvline(model.t_params[i], c=c)
            for j in range(i+1, D):
                ax.axes[j][i].plot(model.t_params[i], model.t_params[j], 'ro')
    return ax

ax = pairplot_truth(df_trace)

# scratch

In [ ]:
from snepits.utils.pymc3 import trace_pairplot

In [ ]:
sns.set_context("paper", font_scale=1.5)

trace_pairplot([df_trace], diag_kind='kde', s=1, t_params=model.t_params)